In [18]:
import requests
from PIL import Image

# Since you're on the same Docker network, use the service name as the host
API_BASE_URL = "http://api:8000"  # Assuming FastAPI runs on port 8000


def load_and_display_image(image_path):
    """Load and display an image in the notebook"""
    img = Image.open(image_path)
    display(img)
    return image_path


def test_image_chat(image_path, query):
    """Test the chat with image endpoint"""
    print(f"Testing with image: {image_path}")
    print(f"Query: {query}")
    
    # Verify image exists and get size
    import os
    if os.path.exists(image_path):
        print(f"Image size: {os.path.getsize(image_path)} bytes")
    else:
        print(f"Warning: Image file not found at {image_path}")
        
    # Prepare the files and data
    files = {"image": ("image.jpg", open(image_path, "rb"), "image/jpeg")}
    data = {
        "query": query,
        "previous_messages": "[]",
    }
    print(f"Request data: {data}")
    
    # Make the request
    url = f"{API_BASE_URL}/chat/with-image"
    try:
        response = requests.post(url, files=files, data=data)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        return f"Error: {str(e)}"


def test_text_chat(query, previous_messages=None):
    """Test the regular chat endpoint"""
    if previous_messages is None:
        previous_messages = []

    data = {"query": query, "messages": previous_messages, "expect_json": False}

    url = f"{API_BASE_URL}/chat"
    try:
        response = requests.post(url, json=data)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        return f"Error: {str(e)}"

text_response = test_text_chat("What is the capital of France?")
print("\nText Chat Response:")
print(text_response)

image_path = "notes-1.png"

image_response = test_image_chat(image_path=image_path, query="""Please convert this image to well formatted markdown with inline and block latex so that it can easily be handled by pandoc""")
print("\nImage Chat Response:")
print(image_response)


Text Chat Response:
{'message': 'The capital of France is Paris.', 'json': None, 'error': None, 'raw_content': None}
Testing with image: notes-1.png
Query: Please convert this image to well formatted markdown with inline and block latex so that it can easily be handled by pandoc
Image size: 270640 bytes
Request data: {'query': 'Please convert this image to well formatted markdown with inline and block latex so that it can easily be handled by pandoc', 'previous_messages': '[]'}

Image Chat Response:
{'message': "Here's the content formatted in markdown with LaTeX:\n\n# Linear Algebra\n\nIn linear algebra we study:\n- Linear systems of equations\n- Matrices\n- etc.\n\nIn abstract or modern algebra topics include:\n- Groups\n- Rings\n- Fields (Finite fields are used in algebraic coding theory)\n- Modules (or [Categories])\n\nMatrix notation: $A = [A_{ij}]_{i,j=1,2,...}$\n\nExample:\n\n$$A = \\begin{bmatrix} \na_{11} & a_{12} \\\\\na_{21} & a_{22}\n\\end{bmatrix}$$\n\nIf $A = A^T$ then $

In [19]:
print(image_response["message"])

Here's the content formatted in markdown with LaTeX:

# Linear Algebra

In linear algebra we study:
- Linear systems of equations
- Matrices
- etc.

In abstract or modern algebra topics include:
- Groups
- Rings
- Fields (Finite fields are used in algebraic coding theory)
- Modules (or [Categories])

Matrix notation: $A = [A_{ij}]_{i,j=1,2,...}$

Example:

$$A = \begin{bmatrix} 
a_{11} & a_{12} \\
a_{21} & a_{22}
\end{bmatrix}$$

If $A = A^T$ then $A$ is square and symmetric.

$$A^T = \begin{bmatrix}
a_{11} & a_{21} \\
a_{12} & a_{22}
\end{bmatrix}$$


In [16]:
import base64
from anthropic import Anthropic
client = Anthropic()
MODEL_NAME = "claude-3-opus-20240229"

def get_base64_encoded_image(image_path):
    with open(image_path, "rb") as image_file:
        binary_data = image_file.read()
        base_64_encoded_data = base64.b64encode(binary_data)
        base64_string = base_64_encoded_data.decode('utf-8')
        return base64_string


message_list = [
    {
        "role": 'user',
        "content": [
            {"type": "image", "source": {"type": "base64", "media_type": "image/png", "data": get_base64_encoded_image("notes-1.png")}},
            {"type": "text", "text": "Transcribe the code in the answer. Only output the code."}
        ]
    }
]

response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=2048,
    messages=message_list
)
print(response.content[0].text)

A = [a11  a12]
    [a21  a22]

If A = AT then
A is square
    and symmetric

Matrix notation: A = [aij], i = 1,2,...
                             j = 1,2,...

Ex)     [a11  a12]
A =     [a12  a22]
